# HuggingFace SmolAgents: Zero to Hero Guide

## Lightweight, Focused, and Code-First AI Agents

**Objective:** This comprehensive notebook takes you from beginner to advanced SmolAgents user. Learn how to build lightweight, efficient agents that write and execute Python code to solve tasks.

**Target Audience:** Software engineers from complete beginners to experts looking to master SmolAgents.

---

## Table of Contents
1. [Introduction & Core Philosophy](#1-introduction--core-philosophy)
2. [Prerequisites & Setup](#2-prerequisites--setup)
3. [Core Concepts: CodeAgent vs ToolCallingAgent](#3-core-concepts-codeagent-vs-toolcallingagent)
4. [Your First Agent](#4-your-first-agent)
5. [Built-in Tools](#5-built-in-tools)
6. [Custom Tools: @tool Decorator](#6-custom-tools-tool-decorator)
7. [Custom Tools: Class-Based](#7-custom-tools-class-based)
8. [Multi-Step Reasoning](#8-multi-step-reasoning)
9. [Agent Memory & State](#9-agent-memory--state)
10. [Streaming Output](#10-streaming-output)
11. [Error Handling & Debugging](#11-error-handling--debugging)
12. [Best Practices & Common Pitfalls](#12-best-practices--common-pitfalls)
13. [Conclusion & Next Steps](#13-conclusion--next-steps)

---

## 1. Introduction & Core Philosophy

### What is SmolAgents?

**SmolAgents** is HuggingFace's lightweight, minimal agent framework. Unlike heavyweight frameworks, SmolAgents focuses on:

- **Simplicity**: Minimal code, easy to understand
- **Code-First**: Agents write Python code, not JSON
- **Flexibility**: Works with any LLM provider
- **Transparency**: See exactly what the agent is thinking

### Core Philosophy

```
┌─────────────────────────────────────────────────────────────────┐
│                SmolAgents Philosophy                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   "Less is more"                                               │
│                                                                 │
│   • Minimal dependencies                                        │
│   • Clear, readable code                                        │
│   • No hidden magic                                            │
│   • Agent writes real Python code                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### When to Use SmolAgents?

✅ **Good for:**
- Learning agent fundamentals
- Lightweight tool-using agents
- Code generation tasks
- Quick prototyping
- When you want full transparency

❌ **Consider alternatives when:**
- You need multi-agent collaboration (use AutoGen/CrewAI)
- You need complex state machines (use LangGraph)
- You need production-ready features out of the box

### SmolAgents vs Other Frameworks

```
┌────────────────────────────────────────────────────────────┐
│              Agent Framework Comparison                    │
├────────────────┬───────────────────────────────────────────┤
│ SmolAgents     │ Minimal, code-first, single agent        │
│ LangChain      │ Full-featured, flexible, tool-calling    │
│ LangGraph      │ State machines, complex control flow     │
│ AutoGen        │ Multi-agent conversations                │
│ CrewAI         │ Role-playing agent crews                 │
└────────────────┴───────────────────────────────────────────┘
```

---

## 2. Prerequisites & Setup

### Requirements

- **Python 3.9+**
- **OpenAI API Key** (or other LLM provider)

### Installation

```bash
pip install smolagents[litellm]
```

The `[litellm]` extra enables using OpenAI, Anthropic, and other providers.

In [ ]:
# Install dependencies (uncomment to run)
# !pip install smolagents[litellm] python-dotenv

In [ ]:
import os
import warnings
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Verify API key
openai_key = os.getenv("OPENAI_API_KEY")

print("🔑 API KEY STATUS")
print("-" * 40)
print(f"OpenAI API Key: {'✅ Found' if openai_key else '❌ Missing'}")

if not openai_key:
    print("\n❌ Please add OPENAI_API_KEY to your .env file")

---

## 3. Core Concepts: CodeAgent vs ToolCallingAgent

SmolAgents provides two main agent types:

### CodeAgent (Recommended)

The agent writes Python code to solve tasks. The code is then executed.

```
┌─────────────────────────────────────────────────────────────────┐
│                    CodeAgent Flow                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query: "What's the weather in Paris?"                   │
│        │                                                        │
│        ▼                                                        │
│   Agent thinks and writes:                                      │
│   ┌─────────────────────────────────────────────┐              │
│   │ weather = get_weather(city="Paris")         │              │
│   │ final_answer(weather)                       │              │
│   └─────────────────────────────────────────────┘              │
│        │                                                        │
│        ▼                                                        │
│   Code is EXECUTED in Python                                   │
│        │                                                        │
│        ▼                                                        │
│   Result returned to user                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Advantages:**
- More flexible (can do math, loops, conditionals)
- Easier to debug (see actual code)
- Can chain multiple tool calls naturally

### ToolCallingAgent

Uses JSON-based tool calls (like OpenAI function calling).

**Advantages:**
- Works better with some models
- More structured output
- Safer (no code execution)

### Key Components

| Component | Description |
|-----------|-------------|
| **Model** | LLM that powers the agent |
| **Tools** | Functions the agent can call |
| **final_answer** | Built-in function to return results |

---

## 4. Your First Agent

Let's create a simple agent that can perform calculations.

In [ ]:
from smolagents import CodeAgent, LiteLLMModel

# Step 1: Initialize the model
# LiteLLMModel wraps any LLM provider
model = LiteLLMModel(model_id="gpt-4o")

print("✅ Model initialized!")
print(f"   Model: gpt-4o")

In [ ]:
# Step 2: Create the agent
# Even without tools, CodeAgent can do math and logic!

agent = CodeAgent(
    model=model,
    tools=[],  # No extra tools, just Python
)

print("✅ Agent created!")

In [ ]:
# Step 3: Run the agent
# The agent will write and execute Python code

print("\n" + "="*60)
print("🚀 RUNNING AGENT")
print("="*60 + "\n")

result = agent.run("Calculate 15 factorial (15!) and tell me how many digits it has")

print("\n" + "="*60)
print("📋 FINAL ANSWER")
print("="*60)
print(result)

### 🎯 Key Observations

Notice how the agent:
1. **Thinks** about the problem
2. **Writes Python code** to solve it
3. **Executes** the code
4. **Returns** the result via `final_answer()`

This is the power of code-first agents!

---

## 5. Built-in Tools

SmolAgents provides several built-in tools you can use immediately.

In [ ]:
from smolagents import (
    CodeAgent, 
    LiteLLMModel,
    DuckDuckGoSearchTool,
    VisitWebpageTool,
)

# List of commonly available tools
print("""
┌─────────────────────────────────────────────────────────────────┐
│              SmolAgents Built-in Tools                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   DuckDuckGoSearchTool   - Web search via DuckDuckGo           │
│   VisitWebpageTool       - Visit and extract webpage content   │
│   PythonInterpreterTool  - Execute Python code                 │
│   SpeechToTextTool       - Convert audio to text               │
│   TextToSpeechTool       - Convert text to audio               │
│                                                                 │
│   Note: Some tools require additional dependencies             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

In [ ]:
# Create an agent with web search capability

search_agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool()],
)

print("✅ Search agent created!")
print(f"   Tools: {[tool.name for tool in search_agent.tools]}")

In [ ]:
# Test the search agent

print("\n" + "="*60)
print("🔍 TESTING WEB SEARCH")
print("="*60 + "\n")

result = search_agent.run("What is the latest version of Python released in 2024?")

print("\n" + "="*60)
print("📋 FINAL ANSWER")
print("="*60)
print(result)

---

## 6. Custom Tools: @tool Decorator

The simplest way to create custom tools is with the `@tool` decorator.

In [ ]:
from smolagents import tool

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.
    
    Args:
        city: The name of the city to get weather for.
    
    Returns:
        A string describing the current weather.
    """
    # Simulated weather data
    weather_data = {
        "paris": "Cloudy, 18°C",
        "london": "Rainy, 12°C",
        "new york": "Sunny, 25°C",
        "tokyo": "Clear, 22°C",
        "sydney": "Warm, 28°C",
    }
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"Weather in {city}: {weather_data[city_lower]}"
    return f"Weather data not available for {city}"

print("✅ Weather tool created!")
print(f"   Name: {get_weather.name}")
print(f"   Description: {get_weather.description[:50]}...")

In [ ]:
@tool
def calculate_tip(bill_amount: float, tip_percentage: float = 15.0) -> str:
    """Calculate the tip amount for a restaurant bill.
    
    Args:
        bill_amount: The total bill amount in dollars.
        tip_percentage: The tip percentage (default 15%).
    
    Returns:
        A string with the tip amount and total.
    """
    tip = bill_amount * (tip_percentage / 100)
    total = bill_amount + tip
    return f"Bill: ${bill_amount:.2f}, Tip ({tip_percentage}%): ${tip:.2f}, Total: ${total:.2f}"

print("✅ Tip calculator tool created!")

In [ ]:
# Create agent with custom tools

custom_agent = CodeAgent(
    model=model,
    tools=[get_weather, calculate_tip],
)

print("✅ Custom tool agent ready!")
print(f"   Tools: {[tool.name for tool in custom_agent.tools]}")

In [ ]:
# Test custom tools

print("\n" + "="*60)
print("🌤️ TESTING WEATHER TOOL")
print("="*60 + "\n")

result = custom_agent.run("What's the weather like in Tokyo?")
print(f"\n📋 Answer: {result}")

In [ ]:
# Test tip calculator

print("\n" + "="*60)
print("💵 TESTING TIP CALCULATOR")
print("="*60 + "\n")

result = custom_agent.run("My dinner bill is $85.50. How much should I tip if I want to tip 20%?")
print(f"\n📋 Answer: {result}")

---

## 7. Custom Tools: Class-Based

For more complex tools with state or initialization, use class-based tools.

In [ ]:
from smolagents.tools import Tool
from typing import ClassVar

class CompanyInfoTool(Tool):
    """A tool to look up information about tech companies."""
    
    # These class variables define the tool's interface
    name: ClassVar[str] = "get_company_info"
    description: ClassVar[str] = "Get information about a tech company including founding year and CEO."
    inputs: ClassVar[dict] = {
        "company_name": {
            "type": "string",
            "description": "The name of the company to look up."
        }
    }
    output_type: ClassVar[str] = "string"
    
    def __init__(self):
        """Initialize with company database."""
        super().__init__()
        # Simulated company database
        self.companies = {
            "google": {"founded": 1998, "ceo": "Sundar Pichai", "hq": "Mountain View, CA"},
            "microsoft": {"founded": 1975, "ceo": "Satya Nadella", "hq": "Redmond, WA"},
            "apple": {"founded": 1976, "ceo": "Tim Cook", "hq": "Cupertino, CA"},
            "amazon": {"founded": 1994, "ceo": "Andy Jassy", "hq": "Seattle, WA"},
            "meta": {"founded": 2004, "ceo": "Mark Zuckerberg", "hq": "Menlo Park, CA"},
        }
    
    def forward(self, company_name: str) -> str:
        """Look up company information."""
        company = company_name.lower().strip()
        if company in self.companies:
            info = self.companies[company]
            return f"{company_name}: Founded {info['founded']}, CEO: {info['ceo']}, HQ: {info['hq']}"
        return f"No information available for {company_name}"

print("✅ CompanyInfoTool class defined!")

In [ ]:
# Another class-based tool example: Inventory Manager

class InventoryTool(Tool):
    """Manage a simple inventory system."""
    
    name: ClassVar[str] = "inventory_manager"
    description: ClassVar[str] = "Check, add, or remove items from inventory. Actions: 'check', 'add', 'remove'"
    inputs: ClassVar[dict] = {
        "action": {
            "type": "string",
            "description": "Action to perform: 'check', 'add', or 'remove'"
        },
        "item": {
            "type": "string",
            "description": "Name of the item"
        },
        "quantity": {
            "type": "integer",
            "description": "Quantity (for add/remove). Optional for check.",
            "nullable": True
        }
    }
    output_type: ClassVar[str] = "string"
    
    def __init__(self):
        super().__init__()
        # Initialize inventory
        self.inventory = {
            "laptop": 10,
            "keyboard": 25,
            "mouse": 50,
            "monitor": 15,
        }
    
    def forward(self, action: str, item: str, quantity: int = None) -> str:
        item_lower = item.lower()
        
        if action == "check":
            if item_lower in self.inventory:
                return f"Inventory: {item} - {self.inventory[item_lower]} units"
            return f"Item '{item}' not in inventory"
        
        elif action == "add":
            if quantity is None:
                return "Error: quantity required for 'add' action"
            if item_lower not in self.inventory:
                self.inventory[item_lower] = 0
            self.inventory[item_lower] += quantity
            return f"Added {quantity} {item}. New total: {self.inventory[item_lower]}"
        
        elif action == "remove":
            if quantity is None:
                return "Error: quantity required for 'remove' action"
            if item_lower not in self.inventory:
                return f"Error: {item} not in inventory"
            if self.inventory[item_lower] < quantity:
                return f"Error: Only {self.inventory[item_lower]} {item} available"
            self.inventory[item_lower] -= quantity
            return f"Removed {quantity} {item}. Remaining: {self.inventory[item_lower]}"
        
        return f"Unknown action: {action}"

print("✅ InventoryTool class defined!")

In [ ]:
# Create agent with class-based tools

class_tool_agent = CodeAgent(
    model=model,
    tools=[CompanyInfoTool(), InventoryTool()],
)

print("✅ Class-based tool agent ready!")
print(f"   Tools: {[tool.name for tool in class_tool_agent.tools]}")

In [ ]:
# Test company info tool

print("\n" + "="*60)
print("🏢 TESTING COMPANY INFO TOOL")
print("="*60 + "\n")

result = class_tool_agent.run("When was Microsoft founded and who is the current CEO?")
print(f"\n📋 Answer: {result}")

In [ ]:
# Test inventory tool

print("\n" + "="*60)
print("📦 TESTING INVENTORY TOOL")
print("="*60 + "\n")

result = class_tool_agent.run("Check how many laptops we have in inventory, then add 5 more")
print(f"\n📋 Answer: {result}")

---

## 8. Multi-Step Reasoning

CodeAgent excels at multi-step tasks because it can write code that chains operations.

In [ ]:
@tool
def get_stock_price(symbol: str) -> float:
    """Get the current stock price for a ticker symbol.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL').
    
    Returns:
        The current stock price.
    """
    # Simulated prices
    prices = {
        "AAPL": 175.50,
        "GOOGL": 140.25,
        "MSFT": 380.00,
        "AMZN": 178.75,
        "META": 485.00,
        "TSLA": 245.30,
    }
    symbol = symbol.upper()
    if symbol in prices:
        return prices[symbol]
    return 0.0

@tool
def calculate_portfolio_value(stocks: dict) -> str:
    """Calculate total portfolio value given stock holdings.
    
    Args:
        stocks: Dictionary of {symbol: shares_owned}.
    
    Returns:
        Breakdown and total portfolio value.
    """
    prices = {
        "AAPL": 175.50,
        "GOOGL": 140.25,
        "MSFT": 380.00,
        "AMZN": 178.75,
        "META": 485.00,
        "TSLA": 245.30,
    }
    
    total = 0
    breakdown = []
    for symbol, shares in stocks.items():
        symbol = symbol.upper()
        if symbol in prices:
            value = prices[symbol] * shares
            total += value
            breakdown.append(f"{symbol}: {shares} shares × ${prices[symbol]:.2f} = ${value:.2f}")
    
    return "\n".join(breakdown) + f"\n\nTotal Portfolio Value: ${total:.2f}"

print("✅ Stock tools created!")

In [ ]:
# Create financial agent

finance_agent = CodeAgent(
    model=model,
    tools=[get_stock_price, calculate_portfolio_value],
)

print("\n" + "="*60)
print("📈 MULTI-STEP REASONING EXAMPLE")
print("="*60 + "\n")

# This requires multiple steps: get prices, then calculate portfolio
result = finance_agent.run("""
I own the following stocks:
- 50 shares of AAPL
- 30 shares of GOOGL  
- 20 shares of MSFT

What is my total portfolio value? Also tell me which stock is worth the most in my portfolio.
""")

print("\n" + "="*60)
print("📋 FINAL ANSWER")
print("="*60)
print(result)

---

## 9. Agent Memory & State

SmolAgents provides basic memory through the `memory` parameter.

In [ ]:
# Create agent with memory enabled

memory_agent = CodeAgent(
    model=model,
    tools=[calculate_tip],
)

print("✅ Memory-enabled agent created!")

In [ ]:
# First interaction
print("\n" + "="*60)
print("💬 CONVERSATION TURN 1")
print("="*60 + "\n")

result1 = memory_agent.run("My restaurant bill is $120")
print(f"\n📋 Answer: {result1}")

In [ ]:
# Second interaction - references previous context
print("\n" + "="*60)
print("💬 CONVERSATION TURN 2 (with context)")
print("="*60 + "\n")

# Note: SmolAgents doesn't have automatic memory like LangChain
# You can pass additional_prompting for context
result2 = memory_agent.run(
    "Calculate a 20% tip for my $120 bill",
)
print(f"\n📋 Answer: {result2}")

### Memory Limitations

SmolAgents is minimalist by design. For complex memory:
- Maintain your own conversation history
- Pass context via `additional_prompting`
- Use LangChain or LangGraph for more advanced memory

---

## 10. Streaming Output

SmolAgents shows progress by default. You can control verbosity.

In [ ]:
# Create agent with different verbosity levels

# Verbose agent (default) - shows step-by-step execution
verbose_agent = CodeAgent(
    model=model,
    tools=[get_weather],
)

print("\n" + "="*60)
print("📡 VERBOSE OUTPUT (default)")
print("="*60 + "\n")

result = verbose_agent.run("What's the weather in Paris?")
print(f"\n📋 Result: {result}")

---

## 11. Error Handling & Debugging

CodeAgent can recover from errors by trying different approaches.

In [ ]:
# Create agent with max_steps limit

safe_agent = CodeAgent(
    model=model,
    tools=[get_weather, calculate_tip],
    max_steps=5,  # Prevent infinite loops
)

print("✅ Safe agent created with max_steps=5")

In [ ]:
# Test error handling - asking about unavailable city

print("\n" + "="*60)
print("⚠️ TESTING ERROR HANDLING")
print("="*60 + "\n")

try:
    result = safe_agent.run("What's the weather in Atlantis?")
    print(f"\n📋 Answer: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Inspect agent logs

print("\n" + "="*60)
print("🔍 INSPECTING AGENT LOGS")
print("="*60 + "\n")

# Run a task and check logs
result = safe_agent.run("Calculate 15% tip on $50")

print("\n📊 Agent Logs:")
if hasattr(safe_agent, 'logs'):
    for i, log in enumerate(safe_agent.logs):
        print(f"  Step {i+1}: {log}")
else:
    print("  (Logs available in verbose output above)")

---

## 12. Best Practices & Common Pitfalls

### ✅ Best Practices

1. **Clear tool descriptions** - The agent only sees the docstring!
2. **Type hints** - Help the agent understand parameters
3. **Return meaningful messages** - Include context in returns
4. **Set max_steps** - Prevent infinite loops
5. **Test tools independently** - Before adding to agent

### ❌ Common Pitfalls

1. **Vague descriptions** - Agent won't know when to use the tool
2. **Missing type hints** - Agent may pass wrong types
3. **Complex nested returns** - Keep returns simple and clear
4. **Too many tools** - Agent gets confused
5. **No error handling in tools** - Crashes break the agent

In [ ]:
# Example: Well-designed tool

@tool
def convert_temperature(value: float, from_unit: str, to_unit: str) -> str:
    """Convert temperature between Celsius, Fahrenheit, and Kelvin.
    
    Args:
        value: The temperature value to convert.
        from_unit: Source unit ('celsius', 'fahrenheit', or 'kelvin').
        to_unit: Target unit ('celsius', 'fahrenheit', or 'kelvin').
    
    Returns:
        A string with the converted temperature.
    
    Example:
        convert_temperature(100, 'celsius', 'fahrenheit') -> '100°C = 212°F'
    """
    # Normalize units
    from_unit = from_unit.lower().strip()
    to_unit = to_unit.lower().strip()
    
    # Convert to Celsius first
    if from_unit == "fahrenheit":
        celsius = (value - 32) * 5/9
    elif from_unit == "kelvin":
        celsius = value - 273.15
    elif from_unit == "celsius":
        celsius = value
    else:
        return f"Error: Unknown unit '{from_unit}'. Use 'celsius', 'fahrenheit', or 'kelvin'."
    
    # Convert from Celsius to target
    if to_unit == "fahrenheit":
        result = celsius * 9/5 + 32
        symbol = "°F"
    elif to_unit == "kelvin":
        result = celsius + 273.15
        symbol = "K"
    elif to_unit == "celsius":
        result = celsius
        symbol = "°C"
    else:
        return f"Error: Unknown unit '{to_unit}'. Use 'celsius', 'fahrenheit', or 'kelvin'."
    
    # Format nicely
    from_symbol = "°F" if from_unit == "fahrenheit" else ("K" if from_unit == "kelvin" else "°C")
    return f"{value}{from_symbol} = {result:.2f}{symbol}"

print("✅ Well-designed temperature converter tool!")

In [ ]:
# Production-ready agent template

def create_production_agent(tools: list, max_steps: int = 10):
    """Create a production-ready SmolAgent with best practices."""
    
    agent = CodeAgent(
        model=LiteLLMModel(model_id="gpt-4o"),
        tools=tools,
        max_steps=max_steps,
    )
    
    return agent

print("✅ Production agent template created!")

---

## 13. Conclusion & Next Steps

### What You've Learned

| Topic | Key Takeaway |
|-------|-------------|
| CodeAgent | Agents that write Python code |
| @tool Decorator | Simplest way to create tools |
| Class-based Tools | For stateful tools |
| Multi-step Reasoning | Chain operations naturally |
| Error Handling | Use max_steps and clear returns |

### When to Choose SmolAgents

✅ Choose SmolAgents when:
- You want minimal dependencies
- You need full transparency
- You're building lightweight agents
- You want code-first approach

❌ Consider alternatives when:
- You need multi-agent systems (AutoGen/CrewAI)
- You need complex state machines (LangGraph)
- You need production-ready memory (LangChain)

### Next Steps

1. **Practice**: Build an agent for your use case
2. **Explore**: Try ToolCallingAgent as an alternative
3. **Compare**: See how LangChain and AutoGen differ
4. **Scale**: Consider LangGraph for complex workflows

### Resources

- [SmolAgents Documentation](https://huggingface.co/docs/smolagents/)
- [SmolAgents GitHub](https://github.com/huggingface/smolagents)
- [HuggingFace Hub](https://huggingface.co/)

---

**Congratulations!** You've completed the SmolAgents Zero to Hero guide! 🎉